In [14]:
import pandas as pd

from config.development import DATABASE

In [16]:
df = pd.read_sql('articles', DATABASE, index_col = 'id')

In [17]:
df.sample(10)

,url,title,date,body,author
id,,,,,
17239,https://www.inform.kz/ru/v-bryussele-obsuzhden...,В Брюсселе обсуждены вопросы обмена таможенной...,2012-06-28,БРЮССЕЛЬ. 28 июня. КАЗИНФОРМ - Председатель Ко...,None
23149,https://www.inform.kz/ru/v-ural-ske-po-lozhnom...,В Уральске по ложному сообщению искали бомбу в...,2012-09-03,УРАЛЬСК. 3 сентября. КАЗИНФОРМ - В Уральске по...,Мейрамбек Байгарин
409,https://www.inform.kz/ru/v-pekine-sostoitsya-t...,В Пекине состоится торжественный прием по случ...,2012-01-09,ПЕКИН. 9 января. КАЗИНФОРМ /Руслан Сулейменов/...,Руслан Сулейменов
31380,https://www.inform.kz/ru/chetyre-bronzy-zavoev...,Четыре «бронзы» завоевали казахстанские боксер...,2012-11-22,АСТАНА. 22 ноября. КАЗИНФОРМ - Четыре бронзовы...,None
8234,https://www.inform.kz/ru/top-menedzher-gruppy-...,Топ-менеджер группы VimpelCom осужден в Алжире,2012-03-29,АСТАНА. 29 марта. КАЗИНФОРМ - Алжирский суд ош...,None
13769,https://www.inform.kz/ru/hozyainu-penthausa-pr...,Хозяину пентхауса пришлось заплатить за парков...,2012-05-24,АСТАНА. 24 мая. КАЗИНФОРМ - Жить в центре горо...,None
29512,https://www.inform.kz/ru/razboynik-iz-kazahsta...,Разбойник из Казахстана на протяжении многих л...,2012-11-06,АЛМАТЫ. 6 ноября. КАЗИНФОРМ - Разбойник из Каз...,Мустафина Сара
1497,https://www.inform.kz/ru/belorussko-kazahstans...,Белорусско-казахстанский товарооборот в 2011 г...,2012-01-18,АСТАНА. 18 января. КАЗИНФОРМ - Белорусско-каза...,None
8811,https://www.inform.kz/ru/v-karagandinskoy-obla...,В Карагандинской области в степи найдены трупы...,2012-04-04,КАРАГАНДА. 4 апреля. КАЗИНФОРМ - В Карагандинс...,None


# Text preprocessing
## Basic preprocessing
* Lower-case
* Remove numbers and non-letter characters
* Lemmatization

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from pymystem3 import Mystem
mystem = Mystem()


def preprocess(text):
    global mystem
    text = text.lower()
    text = re.sub('[\W_\d]+', ' ', text)
    text = mystem.lemmatize(text)
    text = ''.join(text)
    return text

In [18]:
df['body_preprocessed'] = df.body.progress_apply(preprocess)
df['title_preprocessed'] = df.title.progress_apply(preprocess)

  0%|          | 0/32303 [00:00<?, ?it/s]

  0%|          | 0/32303 [00:00<?, ?it/s]

## Search using TF-IDF

In [38]:
tfidf_body = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.8,
    min_df = 3,
    max_features = 10000,
)

tfidf_title = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.8,
    min_df = 3,
    max_features = 5000,
)

tfidf_index = df.index.values
tfidf_body_matrix = tfidf_body.fit_transform(df.body_preprocessed)
tfidf_title_matrix = tfidf_title.fit_transform(df.title_preprocessed)

### Get top __n__ words the article

In [7]:
n, top_n = 1, 5
a_ind = df.sample(n).index
articles = df.body_preprocessed[a_ind]
def top_n_tokens(tfidf_body: TfidfVectorizer, articles: pd.Series, top_n: int) -> list:
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :n]
    m = tfidf_body.get_feature_names()
    
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_ind_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :top_n]
    fn = tfidf_body.get_feature_names()

    return [(indT, [(fn[i], a_tfidf[ind, i]) for i in a_ind_srt[ind]]) for ind, indT in enumerate(articles.index)]

top_n_tokens(tfidf_body, articles, top_n)

[(2294,
  [('сборная', 0.39223057079815954),
   ('матч', 0.3612266905536463),
   ('со счет', 0.2364300432213877),
   ('чемпионат азия', 0.18873499864521773),
   ('победа', 0.177628618367849)])]

In [39]:
from sklearn.metrics.pairwise import linear_kernel

q = "Если хочешь выиграть меня, то сука"

def get_top_n(query, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index, n=10):
    q_transformed = preprocess(q)
    q_transformed_body = tfidf_body.transform([q_transformed])
    q_transformed_title = tfidf_title.transform([q_transformed])

    dist_body = linear_kernel(q_transformed_body, tfidf_body_matrix).flatten()
    dist_title = linear_kernel(q_transformed_title, tfidf_title_matrix).flatten()

    b_weight = 0.3
    t_weight = 1 - b_weight

    dist_weighted = b_weight * dist_body + t_weight * dist_title
    top_n = dist_weighted.argsort()[-n:][::-1]
    return list(zip(tfidf_index[top_n], dist_weighted[top_n]))

get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index)

[(17502, 0.35695989475378637),
 (17403, 0.3365570339254973),
 (10590, 0.2949825758394572),
 (29902, 0.2760671032121855),
 (6498, 0.25999235343192195),
 (27290, 0.25895570023446857),
 (12141, 0.2552753608379427),
 (9166, 0.23654858372284027),
 (29306, 0.23380446277504213),
 (19770, 0.23115299691742297)]

In [2]:
import pickle

In [3]:
SEARCH_FOLDER = './app/models/search/'

In [4]:
pickle.dump(tfidf_body, open(SEARCH_FOLDER + 'tfidf_body.pkl','wb'))
pickle.dump(tfidf_body_matrix, open(SEARCH_FOLDER + 'tfidf_body_matrix.pkl','wb'))
pickle.dump(tfidf_title, open(SEARCH_FOLDER + 'tfidf_title.pkl','wb'))
pickle.dump(tfidf_title_matrix, open(SEARCH_FOLDER + 'tfidf_title_matrix.pkl','wb'))
pickle.dump(tfidf_index, open(SEARCH_FOLDER + 'tfidf_index.pkl','wb'))

NameError: name 'tfidf_body' is not defined

### Load the pickle files for search

In [12]:
del tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix

In [5]:
tfidf_index = pickle.load(open(SEARCH_FOLDER + 'tfidf_index.pkl', 'rb'))
tfidf_body = pickle.load(open(SEARCH_FOLDER + 'tfidf_body.pkl', 'rb'))
tfidf_body_matrix = pickle.load(open(SEARCH_FOLDER + 'tfidf_body_matrix.pkl', 'rb'))
tfidf_title = pickle.load(open(SEARCH_FOLDER + 'tfidf_title.pkl', 'rb'))
tfidf_title_matrix = pickle.load(open(SEARCH_FOLDER + 'tfidf_title_matrix.pkl', 'rb'))

In [43]:
q = 'Олимпиада в пекине'
res = get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index)
ids, ranks = zip(*res)
print(ids)

(6205, 18605, 19599, 20066, 22327, 2247, 30348, 21790, 22225, 15279)


In [31]:
from sklearn.metrics.pairwise import linear_kernel
q = "Если хочешь выиграть меня, то сука"

a = [1001, 1002, 2001, 2002]

q_transformed = preprocess(q)
q_transformed_body = tfidf_body.transform([q_transformed])
q_transformed_title = tfidf_title.transform([q_transformed])

dist_body = linear_kernel(q_transformed_body, tfidf_body_matrix).flatten()
dist_title = linear_kernel(q_transformed_title, tfidf_title_matrix).flatten()

b_weight = 0.3
t_weight = 1 - b_weight

dist_weighted = b_weight * dist_body + t_weight * dist_title

m = np.in1d(tfidf_index, a)
dist_weighted = dist_weighted[m]
index = tfidf_index[m]

top_n = dist_weighted.argsort()[-10:][::-1]

print(index[top_n], dist_weighted[top_n])

[1001 2002 2001 1002] [0.00498131 0.         0.         0.        ]


## Transliteration check

In [1]:
!pip install transliterate

     |████████████████████████████████| 45 kB 282 kB/s eta 0:00:01


In [8]:
from transliterate import translit

translit('я люблю тебя', 'ru')

'я люблю тебя'

In [3]:
translit('dai mne piva', 'ru')

'даи мне пива'

In [4]:
!pip install autocorrect

     |████████████████████████████████| 622 kB 561 kB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.5.0-py3-none-any.whl size=621854 sha256=730dc07d497e901c3e85815f9c9b90dcd48b5041fa1f6ea20a7a8c7a39dcfc69
  Stored in directory: /home/biddy/.cache/pip/wheels/a4/51/6c/f75116aae65b52be7ad1d57e47ad4e89ab818bf45d9093021f
Successfully built autocorrect


In [6]:
from autocorrect import Speller
spell = Speller(lang='ru')
spell("Алматы самыв хуевый город на свете")

dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!


'Алматы самых левый город на свете'

In [7]:
spell('''Пришло теплое лето. На лисной опушки распускаюца колоколчики, незабутки, шыповник. Белые ромашки пратягивают к сонцу свои нежные лепески. Вылитают из уютных гнёзд птинцы. У зверей взраслеет смена. Мидвежата старше всех. Они радились еще холодной зимой в берлоги. Теперь они послушно следуют за строгай матерью. Рыжые лесята весело играют у нары. А кто мелькает в сасновых ветках? Да это лофкие бельчята совершают свои первые высотные прышки. В сумерках выходят на охоту колючии ежата.
Не обижайте лесных малышей. Приходите в лес верными друзями.
(По Н. Надеждиной)
Солнце
Выплыла из-за леса сонце. Повеселела лесная паляна. Капельки расы заиграли в каждом цветке, в каждой травинки.
Но вот набежала тучя и закрыла всё небо. Загрустила природа. Столп пыли полетел к озиру. От реского ветра с деревьев посыпались сухие сучя. Лес глухо и грозно зашумел. На земле поевились мокрые пятна. С халмов потикли ручьи. Удары грома аглушили всю месность. Но граза быстро прошла. И снова над лесом светит солнце.
Ежи
Унас под крыльцом живут ежы. По вечерам вся семья выходит гулять. Взрослые ежи роют землю маленькими лапами. Они достают корешки и едят. Маленкие ежата в это время играют, резвяца.
Аднажды к старому ежу подбежала сабака. Ёж свернулся вклубок и замер. Собака осторожно покатила ежа кпруду. Ёш плюхнулся в воду и поплыл. Я прогнал сабаку. На следующюю весну остался под крыльцом один старый ёжек. Куда девались остальные? Они переселились в другое место. Старый ёж незахотел пакинуть мой дом.
Летнее утро
Я стою возле цветущива клеверного поля. Разноцветный ковёр его переливаеца, меняет окраску. В самой дали сияет ослепителным блеском залатая кайма.Взлетел жаворонок. Серый камочек в первых лучах солнца стал золотым. Яркие искры заиграли вкаплях расы на цветах.Что за чудестные звуки разлились по земле? Это подлетели пчёлы. Они зажужали над чяшей цветов. Проснулся кузнечик. Понеслась и его скрепучая музыка. Теперь всё поле поёт. Все рады тёпламу летниму утру. Радосно и мне.
''')

'Пришло теплое лето. На личной опушки распускают колокольчики, незабудки, шиповник. Белые ромашки притягивают к концу свои нежные лепестки. Вылетают из уютных гнёзд птицы. У зверей взрослеет смена. Медвежата старше всех. Они родились еще холодной зимой в берлоги. Теперь они послушно следуют за строгой матерью. Рыжие лисята весело играют у нары. А кто мелькает в сосновых ветках? Да это ловкие бельчата совершают свои первые высотные прыжки. В сумерках выходят на охоту колючий ежата.\nНе обижайте лесных малышей. Приходите в лес верными друзями.\n(По Н. Надеждой)\n\nСолнце\n\nВыплыла из-за леса сонце. Повеселела лесная поляна. Капельки расы заиграли в каждом цветке, в каждой травинки.\nНо вот набежала тучи и закрыла всё небо. Загрустила природа. Столп пыли полетел к озеру. От резкого ветра с деревьев посыпались сухие судя. Лес глухо и грозно зашумел. На земле появились мокрые пятна. С холмов потекли ручьи. Удары грома заглушили всю месность. Но раза быстро прошла. И снова над лесом светит 